Stock Price Predictor

Description: The project aims to predict future stock prices of a company (e.g., Google) by leveraging a combination of technical indicators, such as the Relative Strength Index (RSI), historical data, & market sentiment analysis derived from news articles. The prediction model employs a Long Short-Term Memory (LSTM) neural network.

In [2]:
import pandas as pd 
import math
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,LSTM
import matplotlib.pyplot as plt